# WNTR Tracer Analysis
In this notebook, we explore **water quality simulations** in WNTR using concepts similar to those covered in Lecture 19.  
We’ll use the `Net3.inp` water network model to demonstrate key examples.

In this exercise, you will:
1. **Define and run** a tracer simulation  
2. **Plot time series** of a tracer at selected junctions  
5. **Visualize spatial patterns** of tracer across the network


## Imports
Install and import WNTR and additional Python packages that are needed for the tutorial
- Numpy is required to define comparison operators (i.e., np.greater) in queries
- Matplotlib is required to create graphics

In [ ]:
# Install required packages if not already available
try:
    import wntr
except ImportError:
    !pip install wntr
    import wntr  # import again after installation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Units
WNTR uses **SI (International System) units (length in meters, time in seconds, mass in kilograms)**.  See https://usepa.github.io/WNTR/units.html for more details.

That means that water age and reaction rates are reported in **s**.

# Tracer Simulation 💧

## Import network model

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
inp = 'networks/Net3.inp'
wn = wntr.network.WaterNetworkModel(inp)
CMS2GPM = 15850.3 # cms to gpm
FT2M = 0.3048 # ft to m

In [ ]:
# check current model settings
wn.options.time.duration/3600
wn.options.time.quality_timestep
wn.options.quality.parameter

## Define and run tracer simulation ⚙️

In [ ]:
wn.options.quality.parameter = 'TRACE'
wn.options.time.quality_timestep = 300 # 5 min
wn.options.time.duration = 24*3600

In [ ]:
# Choose which source to trace (uncomment one)
wn.options.quality.trace_node = 'Lake'   # trace from Lake
# wn.options.quality.trace_node = 'River'  # or trace from River

In [ ]:
# run simulation
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()           

In [ ]:
# get results
# junctions = wn.junction_name_list
nodes_to_plot = ['105', '187', '231','275']  # example node IDs from NET3
trace = results.node['quality'][nodes_to_plot]
trace.head()

## Plot trace at selected nodes

In [ ]:
time_hours = trace.index / 3600  # Convert seconds to hours

plt.plot(time_hours, trace, linewidth = 2, alpha = 0.5)
# Formatting the plot
plt.xlabel('Time (hours)')
plt.ylabel('Age (hr)')
plt.title('Lake Trace')
plt.legend(nodes_to_plot, loc='best') # labels are assigned based on the order that you plot them
plt.show()

## Spatial plot of tracer across the network 🌍

In [ ]:
trace_6 = results.node['quality'].loc[6*3600, :]

In [ ]:
wntr.graphics.plot_network(
    wn,
    node_attribute = trace_6,
    title = 'Lake trace at 6am',
    node_cmap = 'coolwarm',
    node_size = 20,
    link_width = 0.5,
    add_colorbar = True
)
plt.show()

In [ ]:
trace_12 = results.node['quality'].loc[18*3600, :]

In [ ]:
wntr.graphics.plot_network(
    wn,
    node_attribute = trace_12,
    title = 'Lake trace at 12pm',
    node_cmap = 'coolwarm',
    node_size = 20,
    link_width = 0.5,
    add_colorbar = True
)
plt.show()